In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" # suppress info and warning messages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import random
import openai
import json

In [2]:
filename = os.path.join(os.getcwd(), "data", "Engineer_20230826.csv")
df = pd.read_csv(filename, header=0)
df.head()

,RequisitionID,OrigJobTitle,JobTitle,JobDescription
0,NaN,Licensed Stationary Engineer,ENGINEER (all other),Licensed Stationary Engineer \n\n Froedtert So...
1,00000224907,Guidance Navigation and Control (GN&C) Enginee...,ENGINEER (all other),**The Boeing Company** is in search of a **L...
2,00000331804,"Propulsion Engineer - Associate, Mid-Level and...",ENGINEER (all other),"**Job Description**\n\nAt Boeing, we innovate ..."
3,00000336462,Senior Process Controls Engineer,ENGINEER (all other),"**Job Description**\n\nAt Boeing, we innovate ..."
4,00000338951,RF/Microwave Engineer (Level 2 or 3),ENGINEER (all other),"**Job Description**\n\nAt Boeing, we innovate ..."


In [3]:
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Apply the function to the JobDescription column
df['JobDescription'] = df['JobDescription'].apply(clean_html)

C:\Users\mihil\AppData\Local\Temp\ipykernel_25156\3178465879.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


In [4]:
nan_count = np.sum(df.isnull(), axis=0)
nan_count

RequisitionID     1
OrigJobTitle      0
JobTitle          0
JobDescription    0
dtype: int64

In [5]:
random_sample = df['JobDescription'].sample(n=500, random_state=42)
random_sample.head()

9666     SURVIVABILITY ENGINEER 2\n\nLocation:\n\nNewpo...
2724     Requisition Number: 15838 \n\nRequired Travel:...
19006    **Secure our Nation, Ignite your Future**\n\n*...
15304    Description:\n\nWe are looking for an Instrume...
14411    RH2 is currently seeking an entry-level Staff ...
Name: JobDescription, dtype: object

In [6]:
testing_df = df[['JobDescription']][0:4]

In [54]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [7]:
client = openai.OpenAI(api_key = "API_KEY")

def call_openai_api(job_posting): 
    prompt = f"""
    Break up the following job description into the following categories: Marketing, Description, Requirements, Legal.
    For each category, return an array containing the sentences from the job posting corresponding to the category. Return this in JSON formatting

    Job Posting:
    "{job_posting}"

    The format should be as follows:

    **Marketing:** [Any sentence that promotes the job or company]
    **Description:** [Any sentence that explains the job role, company, or purpose of the role]
    **Requirements:** [Any sentence that outlines qualifications or skills needed for the role]
    **Legal:** [Any sentence that includes legal or compliance information such as equal opportunity statements, privacy, or legal disclaimers]
    **Location:** [Any sentence that highlights the location of the position]
    """
    try: 
        chat_completion = client.chat.completions.create( 
        messages=[ { 
            "role": "user", "content": "follow the format and DON't other weird delimiters." + prompt, 
        } ], 
        model="gpt-4o-mini", 
        )

        
        parsed_response =  chat_completion.choices[0].message.content

        '''parsed_response = json.loads(response_content)
        
        file_path = "job_posting_categories.json"

        # Check if the file exists
        if os.path.exists(file_path):
            # If the file exists, read its content
            with open(file_path, "r") as json_file:
                try:
                    existing_data = json.load(json_file)  # Load existing content
                except json.JSONDecodeError:
                    existing_data = []  # If file is empty or invalid JSON, start with an empty list
        else:
            existing_data = []

        # Append the new data to the existing list
        existing_data.append(parsed_response)

        # Write the updated list back to the file
        with open(file_path, "w") as json_file:
            json.dump(existing_data, json_file, indent=4)'''


        return parsed_response
        '''response_dict = chat_completion.choices[0].message.content 

        testing_df['Categorized_Posting']

        save_to_json(response_dict, "output.json")

        return response_dict'''
    
    except Exception as e: 
        return f"Error: {str(e)}"


In [8]:
testing_df['Categorized_Posting'] = testing_df['JobDescription'].apply(call_openai_api)

testing_df

,JobDescription,Categorized_Posting
0,Licensed Stationary Engineer \n\n Froedtert So...,"```json\n{\n ""Marketing"": [\n ""Froed..."
1,**The Boeing Company** is in search of a **L...,"```json\n{\n ""Marketing"": [\n ""Come ..."
2,"**Job Description**\n\nAt Boeing, we innovate ...","```json\n{\n ""Marketing"": [\n ""At Boeing, ..."
3,"**Job Description**\n\nAt Boeing, we innovate ...","```json\n{\n ""Marketing"": [\n ""At Bo..."


In [11]:
for i in range (50):
    raw_string = testing_df['Categorized_Posting'][i]
    testing_df['Categorized_Posting'][i] = raw_string.strip('"').replace('\\n', '\n').replace('\\"', '"').replace("```json", "").replace("```", "").strip()
    
    #pretty print JSON object
    try:
        parsed_json = json.loads(testing_df['Categorized_Posting'][i])
        # Pretty print the JSON object
        formatted_json = json.dumps(parsed_json, indent=4)
        print(formatted_json)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

testing_df.to_csv('classifier_prep.csv', index=False)

{
    "Marketing": [
        "Froedtert South is a comprehensive regional healthcare system that has served southeastern Wisconsin and northern Illinois communities for more than 100 years."
    ],
    "Description": [
        "Duties consist of checking equipment and facilities for proper operation of HVAC systems, pumps, electrical distribution and other essential systems throughout the hospital.",
        "Duties also include the repair and maintenance of the systems.",
        "Performs preventive maintenance and equipment repair on various systems throughout the hospital, which vary in complexity and skill.",
        "Must be able to read English and schematic diagrams in order to troubleshoot and understand systems.",
        "Interprets gauges and readings on various equipment control panels in order to adjust temperatures and settings to the proper levels and conditions.",
        "Must also be able to interpret results of chemical water test and appropriate amounts of chemical

KeyError: 4